In [1]:
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.utils import to_categorical
from keras.layers.merge import concatenate as concat
import numpy as np
from model.vae.vae_fashionmnist import VariationalAutoEncoderMNIST
from util.experiment import Experiment, load_experiments
from util.plotting import plot_label_clusters
import matplotlib.pyplot as plt
import seaborn as sb
import pandas as pd
from collections import OrderedDict
import copy

# Interactive tools
from pathlib import Path
from PIL import Image
import pickle
import numpy as np

import ipywidgets as widgets
from tqdm import tqdm

In [2]:
fashion_mnist = keras.datasets.fashion_mnist
(_, _), (valid_images, valid_labels) = fashion_mnist.load_data()
valid_images = valid_images.astype("float32") / 255.0
valid_images = tf.expand_dims(valid_images, axis=-1)

class_names = ['T-shirt_top', 'Trouser', 'Pullover', 'Dress', 'Coat',
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

2022-06-27 15:55:09.467371: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [5]:
RUN_NAME = "branchedClassifier"
MODEL_NAME = "1416_speak-leave-easy-money"
#MODEL_NAME = "1402_die-continue-entire-room"
EXAMPLE_IMAGE = tf.expand_dims(valid_images[0], axis=0)

In [6]:
experiment = Experiment(name=MODEL_NAME, base_path="experiments/"+RUN_NAME)
base_model = experiment.load_model()
params = {'input_dim': (28, 28, 1), 'z_dim': experiment.params['latent_dim'], 'beta': experiment.params['beta']}
model = VariationalAutoEncoderMNIST.from_saved_model(base_model, params)
model.compile()

print(experiment.params)

{'optimizer': 'Adam', 'learning_rate': 0.001, 'epochs': 30, 'batch_size': 32, 'latent_dim': 10, 'beta': 8}


In [8]:
def forward_pass(img):
    z_mean, z_log_var, _ = model.encode(img)
    z = model.reparameterize(z_mean, z_log_var)
    
    reco = model.decode(z)
    reco = tf.sigmoid(reco)
    
    return z_mean, z_log_var, z, reco

# Forward pass
z_mean, z_log_var, z, reco = forward_pass(EXAMPLE_IMAGE)
stddev = 2*tf.exp(z_log_var)
reco_img = plt.cm.Greys(reco.numpy().squeeze(), bytes=True)
z_tmp = copy.deepcopy(z.numpy())

# Global Sensitivity Measure

- Fix one dimension
- Sample over the other dimensions

You can learn what variables would need to be changed to drive the solution in a given direction or control the system. If your model is exact and the parameters are known, the "standard" methods apply, but if your model is only approximate, a global sensitivity metric may be a better prediction as to how variables cause changes.

You can learn if there are any variables which do not have a true effect on the output. These variables would be practically unidentifiable from data and models can be reduced by removing the terms. It also is predictive as to robustness properties.

You can find ways to automatically sparsify a model by dropping off the components which contribute the least. This matters in automatically generated or automatically detected models, where many pieces may be spurious and global sensitivities would be a method to detect that in a manner that is not sensitive to the chosen parameters.